In [ ]:
# 1.ASSIGNMENT 4
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

# 2. Set up paths
import os
import time
from pathlib import Path

DATASET_PATH = "/kaggle/input/eraser"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")
WEIGHTS_PATH = "runs/train/stationary_detector/weights/best.pt"

# 3. Train YOLOv5 model (wait until done before moving on)
!python train.py --img 640 --batch 8 --epochs 10 --data "{DATA_YAML}" --weights yolov5s.pt --name stationary_detector --cache

# 4. Wait until training finishes and weights are saved
while not os.path.exists(WEIGHTS_PATH):
    print("Waiting for training to complete and weights to be saved...")
    time.sleep(10)

# 5. Load the trained model
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path=WEIGHTS_PATH, force_reload=True)

# 6. Load class names from data.yaml
import yaml
with open(DATA_YAML, 'r') as f:
    data_yaml = yaml.safe_load(f)
class_names = data_yaml['names']

# 7. Define item prices
ITEM_PRICES = {
    "eraser": 5,
    "pencil": 10,
    "scale": 15,
    "sharpener": 7
}

# 8. Function to calculate bill
def calculate_bill(results):
    bill = {}
    total = 0
    for *box, conf, cls in results.xyxy[0]:
        item = class_names[int(cls)]
        bill[item] = bill.get(item, 0) + 1
        total += ITEM_PRICES[item]
    return bill, total

# 9. Run inference on validation images
import cv2
import matplotlib.pyplot as plt

VALID_IMAGES = os.path.join(DATASET_PATH, "valid", "images")
print("\n--- DETECTION RESULTS ON VALIDATION IMAGES ---\n")
for img_file in sorted(os.listdir(VALID_IMAGES))[:5]:  # Show first 5
    img_path = os.path.join(VALID_IMAGES, img_file)
    results = model(img_path)

    # Annotate and show image
    results.render()
    img = results.ims[0]
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(img_file)
    plt.axis('off')
    plt.show()

    # Print bill
    bill, total = calculate_bill(results)
    print(f"🧾 Bill for {img_file}:")
    for item, qty in bill.items():
        print(f"  {item.capitalize()} x {qty} = ₹{ITEM_PRICES[item] * qty}")
    print(f"  Total = ₹{total}\n{'-'*40}")


Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.31 MiB | 33.01 MiB/s, done.
Resolving deltas: 100% (11932/11932), done.
/kaggle/working/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5
Note: you may need to restart the kernel to use updated packages.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-30 17:30:18.442248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746034218.464839     306 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN whe